# Object Designator
Object designator are used to describe objects located in the BulletWorld or the real environment and then resolve them during runtime to concrete objects.

For all following examples we need a BulletWorld, so let's create one.

In [ ]:
from pycram.bullet_world import BulletWorld

world = BulletWorld()

The following publishes the resources to RvizWeb.
(Skip if running locally.)

In [ ]:
from pycram.ros.tf_broadcaster import TFBroadcaster
from pycram.ros.viz_marker_publisher import VizMarkerPublisher

tf_broadcaster = TFBroadcaster()
viz_publisher = VizMarkerPublisher()

Object designators are different from the Object class in bullet_world.py in the way that they can describe multiple instances of similar objects and do not create objects or provide methods to manipulate them. Nethertheless, object designators contain a reference to the BulletWorld object.

An object can be easily described with two parameters.

   * A list of names 
   * A list of types 

In [ ]:
from pycram.bullet_world import Object
from pycram.pose import Pose

Object("kitchen", "environment", "kitchen_no_walls.urdf")

Object("milk", "milk", "milk.stl", pose=Pose([1.3, 1, 0.9]))
Object("another_milk", "milk", "milk.stl", pose=Pose([1.3, 0.8, 0.9]))

Object("froot_loops", "breakfast_cereal", "breakfast_cereal.stl", pose=Pose([1.3, 0.9, 0.95]))
Object("spoon", "cutlery", "spoon.stl", pose=Pose([1.3, 1.1, 0.87]))
Object("bowley_the_bowl", "bowl", "bowl.stl", pose=Pose([-0.8, 1.0, 0.9]))

Since designators only describe an object, we don't really know which specific object the designator is referring to.

In [ ]:
from pycram.designators.object_designator import BelieveObject

# This will yield an ERROR
milk_designator = BelieveObject(types=["milk"])
milk_designator.pose

But as soon as the description is resolved into a specific object, we get its specific position. Here we resolve by querying the all-knowing perception system of the simulation.

In [ ]:
milk_designator = BelieveObject(types=["milk"])
milk_resolved_object = milk_designator.resolve()
print(milk_resolved_object.name, milk_resolved_object.pose)

## Believe Object

Object designators are used to describe objects.

For the start we want an object designator only describing milk. To get all the believed objects of type 'milk', we can create an iterator and ask for the next solution in a lazy way.

In [ ]:
milk_designator = iter(BelieveObject(types=["milk"]))

print(next(milk_designator).name)
print(next(milk_designator).name)

You can also use the type to describe objects, so now we want to have an object designator that describes every food in the world. 

In [ ]:
breakfast_object_designator = \
  iter(BelieveObject(types=["breakfast_cereal", "milk"]))

print(next(breakfast_object_designator).name)
print(next(breakfast_object_designator).name)
print(next(breakfast_object_designator).name)

In certain usecases we want to perform tasks on all solutions of an object designator. We can iterate through every object that fits their description. Here we describe any of the four components for our bowl of cereals. 

In [ ]:
from pycram.designators.object_designator import BelieveObject

object_description = \
  BelieveObject(types=["milk", "breakfast_cereal", "cutlery", "bowl"])

for obj in object_description:
    print(obj.name, "-", obj.type, ":\n", obj.pose, "\n")

## Object Part 
Part of object designators can be used to describe describe something as part of another obeject. For example, you could describe a specific drawer as part of the kitchen. This is necessary since the drawer is no single BulletWorld Object but rather a link of the kitchen which is a BulletWorld Object.

In [ ]:
from pycram.designators.object_designator import ObjectPart

kitchen_desig = BelieveObject(names=["kitchen"]).resolve()

object_description = \
  ObjectPart(names=["sink_area_left_upper_drawer_main"], 
             part_of=kitchen_desig)

print(object_description.resolve())